<a href="https://colab.research.google.com/github/ZinoTAT/Machine-Learning-Practices/blob/master/pytorch/pytorch%E5%BC%A0%E9%87%8F%E8%BF%9B%E9%98%B6%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

1.4.0


# 张量的扩张

## broadcast-免拷贝的自动扩展
当参与运算的量张量尺寸不一致时，一般会自动进行broadcast

In [2]:
x = torch.tensor([1, 2])
y = torch.tensor([[-1], [-2]])
print(x, y)
x1, y1 = torch.broadcast_tensors(x, y)
print(x1, y1)
print(x + y)
print(torch.all(torch.eq((x1 + y1), x + y)))

tensor([1, 2]) tensor([[-1],
        [-2]])
tensor([[1, 2],
        [1, 2]]) tensor([[-1, -1],
        [-2, -2]])
tensor([[ 0,  1],
        [-1,  0]])
tensor(True)


# 张量的拼接

## cat-不增加维度的张量拼接
torch.cat([tensor1, tensor2], dim=index)\
将张量进行拼接，除选择拼接的维度外其他维度必须保持一致

In [3]:
x = torch.rand(4, 3, 32, 32)
y = torch.rand(5, 3, 32, 32)
print(torch.cat([x, y], dim=0).shape)

torch.Size([9, 3, 32, 32])


## stack-增加维度的张量拼接
torch.stack([tensor1, tensor2], dim=index) \
进行拼接的两张量必须尺寸必须相同，拼接后在指定位置增加新的维度

In [4]:
x = torch.rand(4, 3, 32, 32)
y = torch.rand(4, 3, 32, 32)
print(torch.stack([x, y], dim=0).shape)

torch.Size([2, 4, 3, 32, 32])


# 张量的拆分

## split-指定长度的拆分
split(step, dim=index)\
指定一个固定长度，按找长度进行拆分\
split(list, dim=index)\
使用一个list指定每一段拆分的长度进行拆分

In [5]:
a = torch.rand(9, 2)
print(a.split(3, dim=0))
print(a.split([2, 3, 4], dim=0))

(tensor([[0.2389, 0.9716],
        [0.7806, 0.6587],
        [0.1611, 0.8430]]), tensor([[0.8462, 0.3919],
        [0.3506, 0.5595],
        [0.9636, 0.1962]]), tensor([[0.8275, 0.5641],
        [0.3590, 0.8612],
        [0.9973, 0.9838]]))
(tensor([[0.2389, 0.9716],
        [0.7806, 0.6587]]), tensor([[0.1611, 0.8430],
        [0.8462, 0.3919],
        [0.3506, 0.5595]]), tensor([[0.9636, 0.1962],
        [0.8275, 0.5641],
        [0.3590, 0.8612],
        [0.9973, 0.9838]]))


## chunk-指定个数的拆分
chunk(num, dim=index)\
对指定维度拆分成num个新的张量\
在下面的例子中可以看到，当对长度为9的维度进行拆分，指定的拆分个数为4个时，拆分的结果为3个长度为3的张量，而非4个长度和为9的张量，因此我猜测这里拆分的机制应该是以step为单位对长度为len的维度进行拆分，当余下不足step时停止划分，其中step为(len / num)的值向上取整，也就是说：\
chunk(num, dim=index)的结果与split(math.ceil(len / num), dim=index)\
这同时意味着使用chunk划分时划分出的张量个数可能并非我们选择的参数num


In [6]:
a = torch.rand(9, 2)
print(a.chunk(4, dim=0))
print(a.chunk(3, dim=0))

(tensor([[0.5451, 0.6198],
        [0.3198, 0.7075],
        [0.4060, 0.9210]]), tensor([[0.5297, 0.8025],
        [0.6964, 0.5388],
        [0.3651, 0.3266]]), tensor([[0.3271, 0.5128],
        [0.8730, 0.4198],
        [0.7791, 0.7060]]))
(tensor([[0.5451, 0.6198],
        [0.3198, 0.7075],
        [0.4060, 0.9210]]), tensor([[0.5297, 0.8025],
        [0.6964, 0.5388],
        [0.3651, 0.3266]]), tensor([[0.3271, 0.5128],
        [0.8730, 0.4198],
        [0.7791, 0.7060]]))


# 张量的运算
张量的加减运算调用函数方法和使用运算符的结果是一致的，但使用乘号时并不是张量的乘法运算，而是直接将两张量对应位置相乘

## 张量的加减
张量维度不一致时采用从最后一个维度向前对齐并自动broadcast

In [7]:
a = torch.rand(3, 4)
b = torch.rand(4)
print(a + b)
print(torch.all(torch.eq(a + b, torch.add(a, b))))
print(torch.all(torch.eq(a - b, torch.sub(a, b))))

tensor([[1.4007, 0.9844, 1.2676, 0.3802],
        [0.7733, 0.9243, 1.0007, 0.7836],
        [0.9602, 1.3731, 0.5632, 0.6319]])
tensor(True)
tensor(True)


## matmul-向量的乘法
torch.mm可以对两个二维张量进行相乘操作\
torch.matmul可以对不少于二维的张量进行操作，对最后两维做乘法\
matmul等价于运算符@，不等价于运算符*

In [8]:
x = torch.rand(1, 2, 3)
y = torch.rand(1, 3, 4)
print(torch.matmul(x, y).shape)
try:
  x * y
except RuntimeError:
  print('failed')
print(torch.all(torch.eq(torch.matmul(x, y), x@y)))

torch.Size([1, 2, 4])
failed
tensor(True)


## 幂、指数和对数
幂操作可以使用pow()函数或这运算符 **\
指数使用torch.exp(),对数使用torch.log()、torch.log2()、torch.log10()

In [9]:
a = torch.full([2, 3], 2)
print(a)
print(a.pow(2))
print(torch.all(torch.eq(a.pow(3), a**3)))

tensor([[2., 2., 2.],
        [2., 2., 2.]])
tensor([[4., 4., 4.],
        [4., 4., 4.]])
tensor(True)


In [10]:
a = torch.full([2, 3], 3)
print(torch.exp(a))
print(torch.log(a))
print(torch.log2(a))
print(torch.log10(a))

tensor([[20.0855, 20.0855, 20.0855],
        [20.0855, 20.0855, 20.0855]])
tensor([[1.0986, 1.0986, 1.0986],
        [1.0986, 1.0986, 1.0986]])
tensor([[1.5850, 1.5850, 1.5850],
        [1.5850, 1.5850, 1.5850]])
tensor([[0.4771, 0.4771, 0.4771],
        [0.4771, 0.4771, 0.4771]])


## clamp-张量的reLU操作

In [11]:
grad = torch.rand(2, 3) * 10
print(grad)
print(grad.max())
print(grad.median())
print(grad.clamp(5))
print(grad.clamp(5,7))

tensor([[1.2963, 4.1102, 8.6811],
        [1.3078, 6.9858, 3.6912]])
tensor(8.6811)
tensor(3.6912)
tensor([[5.0000, 5.0000, 8.6811],
        [5.0000, 6.9858, 5.0000]])
tensor([[5.0000, 5.0000, 7.0000],
        [5.0000, 6.9858, 5.0000]])


# 张量的属性统计

## norm-范数计算
norm(index, dim= )dim可省略

In [12]:
a = torch.full([2, 3], 1)
print(a.norm(1), a.norm(2), a.norm(2, dim=1))

tensor(6.) tensor(2.4495) tensor([1.7321, 1.7321])


## mean,sum,max,min,prod


In [13]:
a = torch.arange(8).reshape(2, 4).float()
print(a)
print(a.mean(), a.sum(), a.max(dim=0), a.min())
print(a.argmax(),a.argmin(dim=1))

tensor([[0., 1., 2., 3.],
        [4., 5., 6., 7.]])
tensor(3.5000) tensor(28.) torch.return_types.max(
values=tensor([4., 5., 6., 7.]),
indices=tensor([1, 1, 1, 1])) tensor(0.)
tensor(7) tensor([0, 0])
